In [1]:
import time
from time import sleep
from random import randint

import requests
from bs4 import BeautifulSoup as bs 

In [2]:
# 2개 년도의 [제목, 네티즌 평점, 평론가 평점, 누적 관객수] 찾아 리스트에 저장

def daum_movie_crawler(year):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; InteSl Mac O X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    main_url = 'https://movie.daum.net/boxoffice/yearly?year=' + str(year)
    
    # Initialize
    data = []
    cnt= 0 
    
    response = requests.get(main_url,headers=headers)
    body_table = bs(response.text,'html.parser')
    
    # 포스터 테이블 구하기 
    poster_table = body_table.findAll('div', {"class":"wrap_movie"})
    
    # 설명 테이블 구하기
    desc_table = body_table.findAll('div', {"class":"wrap_desc"})

    #포스터 테이블을 looping 
    for poster in poster_table:
        lst = []
        
        # 영화 제목
        title_tag = poster.find('a',{"class":"name_movie #title"})
        lst.append(title_tag.text)
        
        # 네티즌 평점 및 평론가 평점
        grade_tag = poster.findAll('span',{"class":"num_grade"})
        net_grade = float(grade_tag[0].text + "." + grade_tag[2].text[1])
        lst.append(net_grade)
        try:
            cri_grade = float(grade_tag[3].text + "." + grade_tag[5].text[1])
            lst.append(cri_grade)
        except:
            lst.append('NA')
        
        # 누적 관객수
        sub_url = "https://movie.daum.net/moviedb/main/totalAudience.json"
        
        movieId = poster.findAll('a')[0]['href'][13:]
        response = requests.get(sub_url + movieId, headers=headers)
        audience_table = bs(response.text, 'html.parser')
        try:
            audience = int(audience_table.text.split('"')[7].replace(',', ''))
            lst.append(audience)
        except:
            lst.append('NA')
        
        # sleep(random.uniform(0,1))
        data.append(lst)
                                                     
    return data


In [3]:
import pandas as pd

# 크롤러를 통해 얻은 데이터 csv 파일로 저장
def crawling_result():
    df_2018 = pd.DataFrame(daum_movie_crawler(2018))
    df_2019 = pd.DataFrame(daum_movie_crawler(2019))
    data = pd.concat([df_2018, df_2019])
    data.columns = ['title', 'netizen_grade', 'critic_grade', 'num_audience']
    data.to_csv('100개영화정보.csv', encoding='cp949')
    
    return data

In [4]:
df = crawling_result()
df.head()

,title,netizen_grade,critic_grade,num_audience
0,신과함께-인과 연,6.8,6.3,12276115
1,어벤져스: 인피니티 워,7.6,7.3,11212710
2,보헤미안 랩소디,9.0,6.3,9948386
3,미션 임파서블: 폴아웃,8.1,7.7,6584919
4,신과함께-죄와 벌,7.0,6,14411675
